In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV_small.json")
df = sqlContext.read.json("s3n://patricks3db/reviews_Movies_and_TV.json")
df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [2]:
#df.take(5)

In [3]:
ratings = df
ratings = ratings.drop("helpful")
ratings = ratings.drop("reviewText")
ratings = ratings.drop("reviewTime")
ratings = ratings.drop("reviewerName")
ratings = ratings.drop("summary")
ratings = ratings.drop("unixReviewTime")
ratings.columns

['asin', 'overall', 'reviewerID']

In [4]:
ratings.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[asin: string, overall: double, reviewerID: string]

In [5]:
newUserDf = sqlContext.read.json("s3n://patricks3db/mapuidreviewerid/uidreviwerid.json")

In [6]:
newUserDf.take(1)

[Row(reviewerid=u'A1CULZCDCJJGYC', uid=70850)]

In [7]:
newMovieDf = sqlContext.read.json("s3n://patricks3db/mapmidasin/midasin.json")

In [8]:
newMovieDf.take(1)

[Row(asin=u'B000IU36LM', mid=6824)]

In [9]:
newMovieDf.persist(StorageLevel.MEMORY_AND_DISK_SER)
newUserDf.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[reviewerid: string, uid: bigint]

In [10]:
rawRatings1 = ratings.join(newUserDf, ratings.reviewerID == newUserDf.reviewerid)
rawRatings1.printSchema()


root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)



In [11]:
rawRatings2 = rawRatings1.join(newMovieDf, ratings.asin == newMovieDf.asin)
rawRatings2.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerid: string (nullable = true)
 |-- uid: long (nullable = true)
 |-- asin: string (nullable = true)
 |-- mid: long (nullable = true)



In [12]:
#print rawRatings2.take(1)[0].asin

In [13]:
rawRatings3 = rawRatings2.map(lambda row: (row.uid, row.mid, row.overall)).persist(StorageLevel.MEMORY_AND_DISK_SER)

In [14]:
#rawRatings3.take(10)

In [ ]:
#ALS.extractParamMap()

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
rank = 200
numIterations = 30
#model = ALS.train(rawRatings3, rank, numIterations, 0.01)
#model = ALS.trainImplicit(rawRatings3, rank, numIterations, 0.01)
model = ALS.trainImplicit(rawRatings3, rank, numIterations, 0.03)

In [ ]:
model.recommendProducts(100000,5)

In [ ]:
#type(model)

In [ ]:
#type(ALS)

In [ ]:
#userMoviePair = usersDf.join(moviesDf).drop('reviewerID').drop('asin').map(lambda row: (row.userID, row.movieID))

In [ ]:
#userMoviePair.take(5)
#type(userMoviePair)

In [ ]:
#recommendations = model.predictAll(userMoviePair)

In [ ]:
#type(recommendations)

In [ ]:
#recommendations.take(5)

In [ ]:
path = "s3n://patricks3db/modelsComplete100r30i"
model.save(sc, path)

In [ ]:
sameModel = MatrixFactorizationModel.load(sc, path)

In [ ]:
sameModel.recommendProducts(1,5)

In [ ]:
sameModel.userFeatures().persist(StorageLevel.MEMORY_AND_DISK_SER)

In [ ]:
sameModel.recommendProducts(100000,5)

In [ ]:
sameModel.recommendProducts(500000,5)

In [ ]:
sameModel.recommendProducts(1000000,5)